# Distributed Data Classification with NeMo Curator's `QualityClassifier`

This notebook demonstrates the use of NeMo Curator's `QualityClassifier`. The [quality classifier](https://huggingface.co/nvidia/quality-classifier-deberta) is used to classify text as high, medium, or low quality. This helps with data annotation, which is useful in data blending for foundation model training. Please refer to the NemoCurator Quality Classifier DeBERTa Hugging Face page for more information about the quality classifier, including its output labels, here: https://huggingface.co/nvidia/quality-classifier-deberta.

Before running this notebook, please see this [Installation Guide](https://docs.nvidia.com/nemo/curator/latest/admin/installation.html#admin-installation) page for instructions on how to install NeMo Curator. Be sure to use an installation method which includes GPU dependencies.

In [1]:
# Silence Ray logs
import os

os.environ["LOGURU_LEVEL"] = "ERROR"

The following imports are required for this tutorial:

In [ ]:
import pandas as pd

from nemo_curator.core.client import RayClient
from nemo_curator.pipeline import Pipeline
from nemo_curator.stages.text.classifiers import QualityClassifier
from nemo_curator.stages.text.io.reader.jsonl import JsonlReader
from nemo_curator.stages.text.io.writer.jsonl import JsonlWriter

To run a pipeline in NeMo Curator, we must start a Ray cluster. This can be done manually (see the [Ray documentation](https://docs.ray.io/en/latest/ray-core/starting-ray.html)) or with Curator's `RayClient`:

In [3]:
try:
    ray_client = RayClient()
    ray_client.start()
except Exception as e:
    msg = f"Error initializing Ray client: {e}"
    raise RuntimeError(msg) from e

# Initialize Read, Classification, and Write Stages

Functions in NeMo Curator are called stages. For this tutorial, we will initialize 3 stages: a JSONL file reader, a quality classification stage, and a JSONL file writer.

For this tutorial, we will create a sample JSONL file to use:

In [4]:
low_quality_text = """
Volunteering

It's all about the warm, fuzzy feeling when you serve the community, without expectation of gain. Volunteering offers you the necessary experience and development skills to take forward with you, as you venture out to work with other people and apply what you learn, to achieve your career goals.

HOW IT WORKS

SEARCH

BOOK NOW

ENJOY THE SHOW

GET A FREE QUOTE

Planning your event ahead of time is the right move. Contact our experts and let us surprise you.
"""

In [5]:
medium_quality_text = "Traveling to Europe during the off-season can be a more budget-friendly option."

In [6]:
high_quality_text = """
Sharapova has been in New Zealand since well before the New Year, preparing for her 2011 start and requested the opening day match to test her form. "My last tournament was over two months ago and it will be really good to get back playing again."

"My priority since I have been here has been to adjust to time and conditions. I have had a couple of practices a day and think that has been really important."

The three-time Grand Slam champion who once stood number one next plays Voracova after winning their only previous match in 2003.
"""

In [7]:
input_file_path = "./input_data_dir"

# Create sample dataset for the tutorial
text = [low_quality_text, medium_quality_text, high_quality_text]
df = pd.DataFrame({"text": text})

try:
    os.makedirs(input_file_path, exist_ok=True)
    df.to_json(input_file_path + "/data.jsonl", orient="records", lines=True)
except Exception as e:
    msg = f"Error creating input file: {e}"
    raise RuntimeError(msg) from e

We can define the reader stage with:

In [8]:
# Read existing directory of JSONL files
read_stage = JsonlReader(input_file_path, files_per_partition=1)

The classifier stage is broken down under the hood into a tokenizer stage and a model inference stage. Tokenization is run on the CPU while model inference is run on the GPU. Optionally, the classifier predictions may be filtered to include only texts with values listed in `filter_by`.

In [ ]:
# Initialize the quality classifier
classifier_stage = QualityClassifier()

# If desired, you may filter your dataset with:
# classifier_stage = QualityClassifier(filter_by=["High", "Medium"])  # noqa: ERA001
# List of labels: "High", "Medium", "Low"

Finally, we can define a stage for writing the results:

In [10]:
# Write results to a directory
output_file_path = "./quality_classifier_results"
write_stage = JsonlWriter(output_file_path)

# Initialize Pipeline

In NeMo Curator, we use pipelines to run distributed data workflows using Ray. Pipelines take care of resource allocation and autoscaling to achieve enhanced performance and minimize GPU idleness.

For the distributed data classifiers, we are able to achieve speedups by ensuring that model inference is run in parallel across all available GPUs, while other stages such as I/O, tokenization, and filtering are run across all available CPUs. This is possible because Curator pipelines are composable, which allows each stage in a pipeline to run independently and with its own specified hardware resources.

In [11]:
classifier_pipeline = Pipeline(name="classifier_pipeline", description="Run a classifier pipeline")

# Add stages to the pipeline
classifier_pipeline.add_stage(read_stage)
classifier_pipeline.add_stage(classifier_stage)
classifier_pipeline.add_stage(write_stage)

Pipeline(name='classifier_pipeline', stages=[jsonl_reader(JsonlReader), quality_classifier_deberta_classifier(QualityClassifier), jsonl_writer(JsonlWriter)])

Composability is also what allows a classifier to sit between pre-processing and post-processing stages. Typical text pre-processing add-ons include text normalization (lowercasing, URL/email removal, Unicode cleanup) and language identification and filtering (to keep only target languages). A full pipeline may look something like:

```python
pipeline = Pipeline(name="full_pipeline")
pipeline.add_stage(read_stage)                # reader (JSONL/S3/etc.)
pipeline.add_stage(lang_id_stage)             # optional: language filter
pipeline.add_stage(classifier_stage)          # classifier
pipeline.add_stage(write_stage)               # writer (JSONL/Parquet)
```

# Run the  Classifier

Let's run the full classifier pipeline:

In [ ]:
# Run the pipeline
result = classifier_pipeline.run()

Since the pipeline ran to completion and the result was written to a JSONL file, we can shut down the Ray cluster with:

In [ ]:
try:
    ray_client.stop()
except Exception as e:  # noqa: BLE001
    print(f"Error stopping Ray client: {e}")

# Inspect the Output

The write stage returns a list of written files. We can read the output file as a Pandas DataFrame for inspection.

In [14]:
result_file = result[0].data[0]

result_df = pd.read_json(result_file, lines=True)
result_df.head()

,text,quality_pred
0,"\nVolunteering\n\nIt's all about the warm, fuz...",Low
1,Traveling to Europe during the off-season can ...,Medium
2,\nSharapova has been in New Zealand since well...,High


We can see that the predictions were generated as expected.